## Import libararies

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 基本ツール
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import matplotlib.ticker as ticker
import seaborn as sns
import time
import copy

from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  r2_score

# optuna

!pip install optuna
import optuna.integration.lightgbm as lgb
import optuna


     |████████████████████████████████| 296kB 5.7MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 1.2MB 16.4MB/s 
     |████████████████████████████████| 143kB 33.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 112kB 36.2MB/s 
     |████████████████████████████████| 481kB 32.6MB/s 
  Created wheel for alembic: filename=alembic-1.5.5-py2.py3-none-any.whl size=156597 sha256=f2e0ccdb47545abf2af93d7965d609ebcbadaf9f39d6d87d24731bc1bdf61df7
  Stored in directory: /root/.cache/pip/wheels/74/3f/61/7de6e3cef766d2680a5d81b1a388286e640f6a681eb589d643
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=88e5732182e9930afb7eb1e353acf01f86954d4673d11add389695d6863c8d26
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=

## Import dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train
df = pd.read_csv('drive/My Drive/train.csv')

In [ ]:
# Test
df = pd.read_csv('drive/My Drive/test.csv')
df.shape

(200000, 25)

## Split dataset into Categorical and Continuous 

### Train

In [ ]:
# Train
# Categorigal variables + target
df_cat = df.loc[:, ['cat0', 'cat1', 'cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9']]

# Continuous variables + target
df_con = df.loc[:, ['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10','cont11','cont12','cont13','target']]

# Target
y = df['target']

### Test

In [ ]:
# Test
# Categorigal variables + target
df_cat = df.loc[:, ['cat0', 'cat1', 'cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9']]

# Continuous variables + target
df_con = df.loc[:, ['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10','cont11','cont12','cont13']]

## Process Contiuous variables

In [ ]:
df_con = df_con.drop( ['cont2','cont3','cont9','cont12','cont13'], axis=1)

In [ ]:
# def outlier_std(df, thresh=3, columns=None):
#     if columns == None:
#         columns = df.columns
    
#     for col in columns:
#         mean = df[col].mean()
#         std  = df[col].std()
        
#         # 平均値から標準偏差の何倍離れているかを計算
#         border = (np.abs(df[col] - mean)) / std
        
#         # 標準偏差の3倍以上離れているデータを除外
#         df = df[(border < 2.75)]
        
#     return df
  
#   print('Before', df_con.shape)
# df_con = outlier_std(df_con, 3, ['target'])
# print('After ', df_con.shape)

In [ ]:
# print('Before', df_con.shape)
# df_con = outlier_std(df_con, 3, ['target'])
# print('After ', df_con.shape)

In [ ]:
# plt.figure(figsize=(12, 8))
# plt.title('Before')
# sns.distplot(df_con['target'])

In [ ]:
# df_con['target'] = np.log1p(df_con['target'])

In [ ]:
# plt.figure(figsize=(12, 8))
# plt.title('After')
# sns.distplot(df_con['target'])

In [ ]:
# def outlier_std(df, thresh=3, columns=None):
#     if columns == None:
#         columns = df.columns
    
#     for col in columns:
#         mean = df[col].mean()
#         std  = df[col].std()
        
#         # 平均値から標準偏差の何倍離れているかを計算
#         border = (np.abs(df[col] - mean)) / std
        
#         # 標準偏差の3倍以上離れているデータを除外
#         df = df[(border < 2.90)]
        
#     return df
  
# print('Before', merged_data.shape)
# merged_data = outlier_std(merged_data, 2.90, ['target'])
# print('After ', merged_data.shape)

In [ ]:
# df_con = df_con.drop(columns = 'target', axis=1)

In [ ]:
# df_con.info()

## Process Categorical variables

### 要素数を切り口に特徴量を作成

In [ ]:
def create_count_variables(df_cat):

  df_cat_c_variables = df_cat

  # cat4
  df_cat_c_variables['cat4_c_variables'] = np.where((df_cat['cat4'] == 'C') | (df_cat['cat4'] == 'D'), 'others', df_cat['cat4'])

  # cat6
  df_cat_c_variables['cat6_c_variables'] = np.where((df_cat['cat6'] == 'C') | (df_cat['cat6'] == 'D') | (df_cat['cat6'] == 'I') | (df_cat['cat6'] == 'E')
            | (df_cat['cat6'] == 'H') | (df_cat['cat6'] == 'G'), 'others', df_cat['cat4'])

  # cat7
  df_cat_c_variables['cat7_c_variables'] = np.where((df_cat['cat7'] == 'F') | (df_cat['cat7'] == 'A') | (df_cat['cat7'] == 'C') | (df_cat['cat7'] == 'I')
            , 'others', df_cat['cat4'])

  # cat8
  df_cat_c_variables['cat8_c_variables'] = np.where((df_cat['cat8'] == 'F') | (df_cat['cat8'] == 'B'), 'others', df_cat['cat8'])

  # cat9
  df_cat_c_variables['cat9_c_variables'] = np.where((df_cat['cat9'] == 'C') | (df_cat['cat9'] == 'D') | (df_cat['cat9'] == 'E'), 'others', df_cat['cat9'])


  df_cat_c_variables = df_cat_c_variables.drop( ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'], axis=1)


  return df_cat_c_variables

### Target Averageを切り口に特徴量を生成

In [ ]:
def create_t_average_variables(df_cat):

  df_cat_t_average = df_cat

  # cat3
  df_cat_t_average['cat3_t_average'] = np.where((df_cat['cat3'] == 'C') | (df_cat['cat3'] == 'D') | (df_cat['cat3'] == 'B'), 'high', df_cat['cat3'])
  df_cat_t_average['cat3_t_average'] = np.where((df_cat['cat3_t_average'] != 'high'), 'low', df_cat['cat3_t_average'])

  # cat4
  df_cat_t_average['cat4_t_average'] = np.where((df_cat['cat4'] == 'C'), 'high', df_cat['cat4'])
  df_cat_t_average['cat4_t_average'] = np.where((df_cat['cat4_t_average'] != 'high'), 'low', df_cat['cat4_t_average'])

  # cat5
  df_cat_t_average['cat5_t_average'] = np.where((df_cat['cat5'] == 'B') | (df_cat['cat5'] == 'D') | (df_cat['cat5'] == 'A'), 'high', df_cat['cat5'])
  df_cat_t_average['cat5_t_average'] = np.where((df_cat['cat5_t_average'] != 'high'), 'low', df_cat['cat5_t_average'])

  # cat6
  df_cat_t_average['cat6_t_average'] = np.where((df_cat['cat6'] == 'B'), 'high', df_cat['cat6'])
  df_cat_t_average['cat6_t_average'] = np.where((df_cat['cat6_t_average'] != 'high'), 'low', df_cat['cat6_t_average'])

  # cat7
  df_cat_t_average['cat7_t_average'] = np.where((df_cat['cat7'] == 'I'), 'high', df_cat['cat7'])
  df_cat_t_average['cat7_t_average'] = np.where((df_cat['cat7_t_average'] != 'high'), 'low', df_cat['cat7_t_average'])

  # cat8
  df_cat_t_average['cat8_t_average'] = np.where((df_cat['cat8'] == 'C') | (df_cat['cat8'] == 'E') | (df_cat['cat8'] == 'A') | (df_cat['cat8'] == 'D') | (df_cat['cat8'] == 'F') | (df_cat['cat8'] == 'B'), 'high', df_cat['cat8'])
  df_cat_t_average['cat8_t_average'] = np.where((df_cat['cat8_t_average'] != 'high'), 'low', df_cat['cat8_t_average'])

  # cat9
  df_cat_t_average['cat9_t_average'] = np.where((df_cat['cat9'] == 'L') | (df_cat['cat9'] == 'G') | (df_cat['cat9'] == 'C') | (df_cat['cat9'] == 'E'), 'high', df_cat['cat9'])
  df_cat_t_average['cat9_t_average'] = np.where((df_cat['cat9_t_average'] != 'high'), 'low', df_cat['cat9_t_average'])

  df_cat_t_average = df_cat_t_average.drop( ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'], axis=1)

  return df_cat_t_average

In [ ]:
df_cat_c_variables = create_count_variables(df_cat)
df_cat_c_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   cat4_c_variables  200000 non-null  object
 1   cat6_c_variables  200000 non-null  object
 2   cat7_c_variables  200000 non-null  object
 3   cat8_c_variables  200000 non-null  object
 4   cat9_c_variables  200000 non-null  object
dtypes: object(5)
memory usage: 7.6+ MB


In [ ]:
df_cat_t_average = create_t_average_variables(df_cat)
df_cat_t_average.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   cat4_c_variables  200000 non-null  object
 1   cat6_c_variables  200000 non-null  object
 2   cat7_c_variables  200000 non-null  object
 3   cat8_c_variables  200000 non-null  object
 4   cat9_c_variables  200000 non-null  object
 5   cat3_t_average    200000 non-null  object
 6   cat4_t_average    200000 non-null  object
 7   cat5_t_average    200000 non-null  object
 8   cat6_t_average    200000 non-null  object
 9   cat7_t_average    200000 non-null  object
 10  cat8_t_average    200000 non-null  object
 11  cat9_t_average    200000 non-null  object
dtypes: object(12)
memory usage: 18.3+ MB


### ダミー変数化

### カウント数を切り口にした場合

In [ ]:
num = [4,6,7,8,9]
for n in range(5):
  df_cat_t_average = pd.get_dummies(df_cat_t_average, columns=['cat'+str(num[n])+'_c_variables'], drop_first=True)

print('done')

done


### Target Averageを切り口にした場合

In [ ]:
for n in range(3,10):
    df_cat_t_average = pd.get_dummies(df_cat_t_average, columns=['cat'+str(n)+'_t_average'], drop_first=True)
print('done')

done


In [ ]:
df_cat_t_average.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   cat4_c_variables_B       200000 non-null  uint8
 1   cat4_c_variables_others  200000 non-null  uint8
 2   cat6_c_variables_B       200000 non-null  uint8
 3   cat6_c_variables_C       200000 non-null  uint8
 4   cat6_c_variables_D       200000 non-null  uint8
 5   cat6_c_variables_others  200000 non-null  uint8
 6   cat7_c_variables_B       200000 non-null  uint8
 7   cat7_c_variables_C       200000 non-null  uint8
 8   cat7_c_variables_D       200000 non-null  uint8
 9   cat7_c_variables_others  200000 non-null  uint8
 10  cat8_c_variables_C       200000 non-null  uint8
 11  cat8_c_variables_D       200000 non-null  uint8
 12  cat8_c_variables_E       200000 non-null  uint8
 13  cat8_c_variables_G       200000 non-null  uint8
 14  cat8_c_variables_others  200000 non-

### 結合

In [ ]:
print('df_cat', df_cat_t_average.shape)
print('df_con', df_con.shape)

df_cat (200000, 34)
df_con (200000, 9)


In [ ]:
merged_data = pd.concat([df_cat_t_average, df_con], axis=1)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 43 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cat4_c_variables_B       200000 non-null  uint8  
 1   cat4_c_variables_others  200000 non-null  uint8  
 2   cat6_c_variables_B       200000 non-null  uint8  
 3   cat6_c_variables_C       200000 non-null  uint8  
 4   cat6_c_variables_D       200000 non-null  uint8  
 5   cat6_c_variables_others  200000 non-null  uint8  
 6   cat7_c_variables_B       200000 non-null  uint8  
 7   cat7_c_variables_C       200000 non-null  uint8  
 8   cat7_c_variables_D       200000 non-null  uint8  
 9   cat7_c_variables_others  200000 non-null  uint8  
 10  cat8_c_variables_C       200000 non-null  uint8  
 11  cat8_c_variables_D       200000 non-null  uint8  
 12  cat8_c_variables_E       200000 non-null  uint8  
 13  cat8_c_variables_G       200000 non-null  uint8  
 14  cat8

In [ ]:
def outlier_std(df, thresh=3, columns=None):
    if columns == None:
        columns = df.columns
    
    for col in columns:
        mean = df[col].mean()
        std  = df[col].std()
        
        # 平均値から標準偏差の何倍離れているかを計算
        border = (np.abs(df[col] - mean)) / std
        
        # 標準偏差の3倍以上離れているデータを除外
        df = df[(border < 3.00)]
        
    return df
  
print('Before', merged_data.shape)
merged_data = outlier_std(merged_data, 3.00, ['target'])
print('After ', merged_data.shape)

Before (200000, 43)


KeyError: ignored

In [ ]:
y = merged_data['target']

KeyError: ignored

In [ ]:
merged_data = merged_data.drop(columns = 'target', axis=1)

KeyError: ignored

### Scaling

In [ ]:
merged_data.shape

(200000, 43)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()
data = mm.fit_transform(merged_data)
data.shape

(200000, 43)

In [ ]:
X = merged_data
X.shape

(200000, 43)

In [ ]:
y.shape

(299573,)

In [ ]:
# 時間計測

def tic():
    # require to import time
    global start_time_tictoc
    start_time_tictoc = time.time()
    
    
def toc(tag = 'elapsed time'):
    if 'start_time_tictoc' in globals():
        print('{}: {:.5f} sec'.format(tag, time.time() - start_time_tictoc), end = '' )
    else:
        print('tic has not been called')

In [ ]:
# 交差検証のためのtrainデータとtestデータの作成

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=1)


# lgbのデータセットに変更

trains = lgb.Dataset(train_x,train_y)
tests  = lgb.Dataset(test_x, test_y)

In [ ]:
def objective(trial):
    params = {
        'random_state': 1,
        'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 10),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7, 0.8,0.9,1]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1,0.2,0.3,0.4]),
    }
    model = XGBRegressor(**params) 
    model.fit(train_x, train_y, eval_set=[(test_x,test_y)], early_stopping_rounds=100, verbose=0)
    y_pred = model.predict(test_x)
    rmse = mean_squared_error(test_y, y_pred, squared=False)
    
    return rmse

print('DONE')

DONE


In [ ]:
# 最適値を見つける

tic()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

toc()


print('Best parameters:', study.best_trial.params)
print('Best RMSE:', study.best_trial.value)

[I 2021-02-28 03:08:41,610] A new study created in memory with name: no-name-db50937b-dd35-4802-9360-1211ddb221c8


[03:08:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-28 03:10:57,841] Trial 0 finished with value: 0.8521379799659278 and parameters: {'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.22809339559349534, 'reg_lambda': 9.151979725287898, 'reg_alpha': 9.358553825215157, 'gamma': 7.353895725973889, 'subsample': 0.8, 'colsample_bytree': 0.2}. Best is trial 0 with value: 0.8521379799659278.


[03:10:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-28 03:11:31,890] Trial 1 finished with value: 0.8553384891219165 and parameters: {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.9070158948702112, 'reg_lambda': 4.441769201581694, 'reg_alpha': 5.371561452016774, 'gamma': 6.827227461631546, 'subsample': 1, 'colsample_bytree': 0.4}. Best is trial 0 with value: 0.8521379799659278.


...

[03:49:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-28 03:49:43,550] Trial 28 finished with value: 10.78271899756911 and parameters: {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 2.5450429751618904, 'reg_lambda': 7.337631097143532, 'reg_alpha': 8.404175515814277, 'gamma': 9.002208439158192, 'subsample': 0.6, 'colsample_bytree': 0.4}. Best is trial 5 with value: 0.8517816527452119.


[03:49:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-28 03:50:32,384] Trial 29 finished with value: 0.8531452883200841 and parameters: {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.4615187591746796, 'reg_lambda': 3.84486275090437, 'reg_alpha': 4.752109585430225, 'gamma': 7.806023602942261, 'subsample': 0.8, 'colsample_bytree': 0.1}. Best is trial 5 with value: 0.8517816527452119.


elapsed time: 2510.77716 secBest parameters: {'n_estimators': 1000, 'max_depth': 8, 'learning_rate': 0.01363011168228136, 'reg_lambda': 4.646647431984033, 'reg_alpha': 5.2605899078939, 'gamma': 5.64993441680361, 'subsample': 0.6, 'colsample_bytree': 0.4}
Best RMSE: 0.8517816527452119


In [ ]:
study.best_params

{'colsample_bytree': 0.4,
 'gamma': 5.64993441680361,
 'learning_rate': 0.01363011168228136,
 'max_depth': 8,
 'n_estimators': 1000,
 'reg_alpha': 5.2605899078939,
 'reg_lambda': 4.646647431984033,
 'subsample': 0.6}

### 予測


In [ ]:
params = study.best_params
params['random_state'] = 0
params['n_estimators'] = 10000## 

In [ ]:
# 予測
X_test = X

model = XGBRegressor(**params)

model.fit(train_x, train_y,
           eval_set =[(train_x, train_y), (test_x, test_y)],
           eval_metric="rmse", verbose=200, early_stopping_rounds=100
           )

    
pred = model.predict(X_test)

[04:15:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:6.92176	validation_1-rmse:6.92119
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[200]	validation_0-rmse:0.968715	validation_1-rmse:0.969009
[400]	validation_0-rmse:0.852611	validation_1-rmse:0.856286
[600]	validation_0-rmse:0.847552	validation_1-rmse:0.853382
[800]	validation_0-rmse:0.844932	validation_1-rmse:0.852324
[1000]	validation_0-rmse:0.843097	validation_1-rmse:0.851745
[1200]	validation_0-rmse:0.841709	validation_1-rmse:0.851464
[1400]	validation_0-rmse:0.840523	validation_1-rmse:0.851242
[1600]	validation_0-rmse:0.839495	validation_1-rmse:0.851108
[1800]	validation_0-rmse:0.838516	validation_1-rmse:0.851056
[2000]	validation_0-rmse:0.837682	validation_1-rmse:0.851025
[2200]	validation_0-rmse:0.836865	validation_1-rmse

In [ ]:
pred.shape

(200000,)

In [ ]:
sample_submission = pd.read_csv('drive/My Drive/sample_submission.csv')
sample_submission.head()

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5


In [ ]:
sample_submission['target'] = pred
sample_submission.to_csv('sample_submissioin.csv', index=False)
sample_submission

,id,target
0,0,7.483742
1,5,7.675493
2,15,7.396656
3,16,7.307775
4,17,7.128921
...,...,...
199995,499987,7.565831
199996,499990,7.410156
199997,499991,7.522186
199998,499994,7.371075


In [ ]:
from google.colab import files
sample_submission.to_csv("output_3.00.csv")
files.download('output_3.00.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>